In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
from RBM import *
from principal_DNN_mnist import *
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
from keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Reshape data to fit input shape of model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])

# Turn images to binary images
X_train = X_train > 127
X_test  = X_test > 127

#Original label to one hot coding label
label_test = np.zeros((Y_test.shape[0],10))
label_train = np.zeros((Y_train.shape[0],10))

for i in range(len(Y_train)):
    label_train[i, Y_train[i]] = 1

for i in range(len(Y_test)):
    label_test[i, Y_test[i]] = 1

Y_train = label_train
Y_test = label_test

2022-04-17 16:47:51.865111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-17 16:47:51.865130: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# HERE TO CHOOSE SIZE OF TRAINING SET

Original size of MNIST is 60000 in training set and 10000 in test set.

Project asks us to test our model in training set of increasing size: 1000, 3000. 7000, 10000, 30000, 60000  

So choosing training_size to have these sizes: 1/60, 1/20, 7/60, 1/6, 1/2, 1.

In [5]:
training_size = 1.0

In [6]:
test_size = 1.0
# Data processing
train_idx = np.random.randint(0, X_train.shape[0], int(X_train.shape[0]*training_size))
test_idx = np.random.randint(0, X_test.shape[0], int(X_test.shape[0]*test_size))

X_train = X_train[train_idx]
Y_train = Y_train[train_idx]

X_test  = X_test[test_idx]
Y_test  = Y_test[test_idx]

#########

X_train = torch.from_numpy(X_train).to(device).float()
Y_train = torch.from_numpy(Y_train).to(device).float()
X_test = torch.from_numpy(X_test).to(device).float()
Y_test = torch.from_numpy(Y_test).to(device).float()

# HERE TO CHOOSE SIZE OF MODEL 

Size of model to be modify according to the project is number of neurons in each layer and number of layers.

In [7]:
#hidden_layers = [150, 200] # To be modified. This is list of number of nodes in each layer (each value is a hidden layer) 

In [8]:
# Setting model hyper-parameters
nb_epochs_pretrain = 200
nb_epochs_scratch = 100
lr = 0.1
batch_size = 64
input_shape = X_train.shape[1]
nb_class = Y_train.shape[1]

###### Training method 1: Construct a DNN model. First train it unsupervisedly and then fine-tune with supervised backpropagation 

In [9]:
#DNN_method_1 = DNN(input_shape, nb_class, hidden_layers)

In [10]:
#DNN_method_1.pretrain_DNN(X_train, nb_epochs_pretrain, lr, batch_size)

In [11]:
#DNN_method_1, loss = retropropagation(DNN_method_1, input = X_train, label = Y_train, epochs = nb_epochs_scratch, lr = lr, batch_size = batch_size)

In [12]:
#accuracy_method_1 = test_DNN(DNN_method_1, X_test, Y_test)

###### Training method 2: Construct a DNN model. Train it with supervised backpropagation from scratch

In [13]:
#DNN_method_2 = DNN(input_shape, nb_class, hidden_layers)

In [14]:
#DNN_method_2, loss = retropropagation(DNN_method_2, input = X_train, label = Y_train, epochs = nb_epochs_scratch, lr = lr, batch_size = batch_size)

In [15]:
#accuracy_method_2 = test_DNN(DNN_method_2, X_test, Y_test)

### For each change of a hyperparameter of the model, we store accuracy_method_1 and accuracy_method_2 to plot these values in relation with chosen hyperparameter.

Note: For this implementation of pytorch in model, sometimes, the accuracy is 0. I don't know why after a lot of research on the code (it seems that there is problem with weight initialization). In that case, re-train the model and calculate the accuracy.

In [16]:
hidden_layers = [[200], [200, 200], [200, 200, 200], [200, 200, 200, 200], [200, 200, 200, 200, 200], [200, 200, 200, 200, 200, 200, 200]]

In [17]:
result = {}

In [18]:
for hid_layer in hidden_layers:
    acc = []
    print(f"Launching for hidden layers: {hid_layer}")
    DNN_method_1 = DNN(input_shape, nb_class, hid_layer)
    DNN_method_1.pretrain_DNN(X_train, nb_epochs_pretrain, lr, batch_size)
    DNN_method_1, loss = retropropagation(DNN_method_1, input = X_train, label = Y_train, epochs = nb_epochs_scratch, lr = lr, batch_size = batch_size)
    print("Method 1")
    accuracy_method_1 = test_DNN(DNN_method_1, X_test, Y_test)
    acc.append(accuracy_method_1[1])
    print("Method 2")
    DNN_method_2 = DNN(input_shape, nb_class, hid_layer)
    DNN_method_2, loss = retropropagation(DNN_method_2, input = X_train, label = Y_train, epochs = nb_epochs_scratch, lr = lr, batch_size = batch_size)
    accuracy_method_2 = test_DNN(DNN_method_2, X_test, Y_test)
    acc.append(accuracy_method_2[1])
    result[f"{len(hid_layer)} layers of size {hid_layer[0]}"]=acc
    print("---------------")

Launching for hidden layers: [200]
Training layer 0
Training Started. 200 iterations.


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [02:44<00:00,  1.21it/s]


End Training
--------------


100%|███████████████████████████████████████████████| 100/100 [00:29<00:00, 3.38it/s, loss =0.00009]


Method 1
Accuracy: 0.9709
Method 2


100%|███████████████████████████████████████████████| 100/100 [00:28<00:00, 3.47it/s, loss =0.09995]


Accuracy: 0.9114
---------------
Launching for hidden layers: [200, 200]
Training layer 0
Training Started. 200 iterations.


  8%|██████▏                                                                           | 15/200 [00:12<02:32,  1.21it/s]


KeyboardInterrupt: 

In [24]:
print(result)

{0.016666666666666666: [0.893375, 0.844], 0.05: [0.93425, 0.90725]}


In [20]:
import json
with open("result_datasize_change.json", "w") as outfile:
    json.dump(result, outfile)